In [ ]:
import pandas as pd
import time
from openai import OpenAI
client = OpenAI(
    api_key="your-api-key",  
    base_url="https://api.nuwaapi.com/v1"  
)

df = pd.read_csv("adverse_event_frequencies.csv")
ae_list = df["Adverse Event"].astype(str).tolist()

fixed_categories = [
    "Neuropsychiatric Disorders",
    "Neuromuscular and Sensory Disorders",
    "Gastrointestinal and Hepatic Disorders",
    "Metabolic and Endocrine Disorders",
    "Dermatological and Hypersensitivity Reactions",
    "Cardiovascular and Circulatory Disorders",
    "Respiratory Disorders",
    "Reproductive and Urogenital Disorders",
    "Renal and Electrolyte Disorders",
    "Hematological and Lymphatic Disorders",
    "Immune and Autoimmune Disorders",
    "Opportunistic Infections",
    "Antiviral Treatment Challenges",
    "Pregnancy and Birth Complications",
    "Congenital and Developmental Disorders",
    "General Symptoms and Fatigue",
    "Diagnostic and Therapeutic Procedures",
    "Injury and Procedural Complications",
    "Other"
]

gpt_results = []

for ae in ae_list:
    prompt = f"""
You are a medical classification expert. Your task is to assign the following adverse event to one of the 19 fixed categories listed below.

You must choose ONLY ONE category. Do not explain or list multiple options. Only return the category name exactly as written.

Categories:
{chr(10).join(fixed_categories)}

Adverse Event: {ae}

Respond ONLY with the most appropriate category name from the list above.
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a medical classification expert."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,
            max_tokens=20
        )

        category = response.choices[0].message.content.strip()
        gpt_results.append(category)
        print(f"{ae} → {category}")

    except Exception as e:
        gpt_results.append("Uncategorized")
        print(f" Error on {ae}: {str(e)}")

    time.sleep(1.2)

df["GPT_Category"] = gpt_results
df.to_csv("adverse_event_classified_by_gpt.csv", index=False)